In [ ]:
# 这会将你的Google Drive挂载到Colab虚拟机上
from google.colab import drive
drive.mount('/content/drive')

# 待办：在你的Drive中输入保存了解压后的作业文件夹的路径，
# 例如 'cs231n/assignments/assignment3/'
FOLDERNAME = 'cs231n/assignments/assignment3/'
assert FOLDERNAME is not None, "[!] 请输入文件夹名称。"

# 现在我们已经挂载了你的Drive，这能确保
# Colab虚拟机的Python解释器可以从其中加载
# Python文件
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# 这会将COCO数据集下载到你的Drive中
# 如果它还不存在的话
%cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
# !bash get_datasets.sh  # 获取数据集的bash脚本（注释掉了）
!bash get_coco_dataset.sh  # 执行获取COCO数据集的bash脚本
%cd /content/drive/My\ Drive/$FOLDERNAME  # 切换回作业文件夹目录


# 使用Transformer进行图像 captioning（图像描述生成）

你现在已经实现了一个基础的循环神经网络（RNN），用于图像描述生成任务。在本笔记本中，你将实现Transformer解码器的关键部分，以完成同样的任务。

**注意：** 与RNN笔记本不同，本笔记本将主要使用PyTorch而非NumPy编写。

In [ ]:
# 环境设置单元格
import time, os, json  # 导入时间、操作系统、JSON处理相关模块
import numpy as np  # 导入NumPy库，用于数值计算
import matplotlib.pyplot as plt  # 导入Matplotlib库，用于绘图

# 从cs231n模块导入相关工具
from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array  # 梯度检查工具
from cs231n.transformer_layers import *  # Transformer层相关组件
from cs231n.captioning_solver_transformer import CaptioningSolverTransformer  # Transformer图像描述生成求解器
from cs231n.classifiers.transformer import CaptioningTransformer  # 基于Transformer的图像描述生成模型
from cs231n.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions  # COCO数据集处理工具
from cs231n.image_utils import image_from_url  # 从URL获取图像的工具

# 设置Matplotlib在 notebook 中内嵌显示
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10.0, 8.0)  # 设置图像默认大小
plt.rcParams['image.interpolation'] = 'nearest'  # 设置图像插值方式为最近邻
plt.rcParams['image.cmap'] = 'gray'  # 设置默认颜色映射为灰度

%load_ext autoreload  # 加载自动重载扩展
%autoreload 2  # 设置自动重载模式，修改文件后自动重新导入

def rel_error(x, y):
    """计算相对误差"""
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))  # 避免除以零的相对误差计算

# COCO数据集

和之前的笔记本一样，我们将使用COCO数据集来进行图像描述生成任务。

In [ ]:
# 从磁盘加载COCO数据到一个字典中（使用PCA降维后的特征）
data = load_coco_data(pca_features=True)

# 打印数据字典中的所有键和对应的值信息
for k, v in data.items():
    if type(v) == np.ndarray:  # 如果值是NumPy数组
        # 打印键名、值的类型、数组形状和数据类型
        print(k, type(v), v.shape, v.dtype)
    else:  # 如果值不是NumPy数组
        # 打印键名、值的类型和长度
        print(k, type(v), len(v))

# Transformer（转换器）

如你所见，循环神经网络（RNN）虽然功能极其强大，但训练速度往往很慢。此外，RNN很难对长距离依赖关系进行编码（不过长短期记忆网络（LSTM）是缓解这一问题的一种方法）。2017年，Vaswani等人在他们的论文["Attention Is All You Need"](https://arxiv.org/abs/1706.03762) 中提出了Transformer，其目的一是引入并行性，二是让模型能够学习长距离依赖关系。这篇论文不仅催生了自然语言处理领域中像BERT和GPT这样著名的模型，还在包括计算机视觉在内的多个领域引发了研究热潮。虽然我们在这里是在图像描述生成的背景下介绍该模型，但注意力机制本身的理念具有更为广泛的适用性。


# Transformer：多头注意力机制

### 点积注意力

回想一下，注意力可以看作是对一个查询向量$q\in\mathbb{R}^d$、一组值向量$\{v_1,\dots,v_n\}, v_i\in\mathbb{R}^d$以及一组键向量$\{k_1,\dots,k_n\}, k_i \in \mathbb{R}^d$执行的一种操作，具体定义为

\begin{align}
c &= \sum_{i=1}^{n} v_i \alpha_i \\
\alpha_i &= \frac{\exp(k_i^\top q)}{\sum_{j=1}^{n} \exp(k_j^\top q)} \\
\end{align}

其中$\alpha_i$通常被称为“注意力权重”，而输出$c\in\mathbb{R}^d$是对应的值向量的加权平均值。

### 自注意力
在Transformer中，我们会执行自注意力操作，这意味着值向量、键向量和查询向量都来自输入$X \in \mathbb{R}^{\ell \times d}$，其中$\ell$是我们的序列长度。具体来说，我们会学习参数矩阵$V,K,Q \in \mathbb{R}^{d\times d}$，并按如下方式对输入$X$进行映射：

\begin{align}
v_i = Vx_i\ \ i \in \{1,\dots,\ell\}\\
k_i = Kx_i\ \ i \in \{1,\dots,\ell\}\\
q_i = Qx_i\ \ i \in \{1,\dots,\ell\}
\end{align}

### 多头缩放点积注意力

在多头注意力的情况下，我们为每个注意力头学习一个参数矩阵，这能让模型拥有更强的表达能力，从而关注输入的不同部分。设$h$为注意力头的数量，$Y_i$为第$i$个注意力头的输出。因此，我们会学习各个头对应的矩阵$Q_i$、$K_i$和$V_i$。为了使整体计算量与单头注意力的情况保持一致，我们将$Q_i$、$K_i$和$V_i$的维度设定为$Q_i \in \mathbb{R}^{d\times d/h}$、$K_i \in \mathbb{R}^{d\times d/h}$和$V_i \in \mathbb{R}^{d\times d/h}$。在上述简单点积注意力的基础上加入缩放因子$\frac{1}{\sqrt{d/h}}$，就得到了（多头缩放点积注意力）：

\begin{equation} 
Y_i = \text{softmax}\bigg(\frac{(XQ_i)(XK_i)^\top}{\sqrt{d/h}}\bigg)(XV_i)
\end{equation}

其中$Y_i\in\mathbb{R}^{\ell \times d/h}$，$\ell$为我们的序列长度。

在我们的实现中，会对注意力权重应用dropout（不过在实际应用中，它可以在任何步骤中使用）：

\begin{equation} 
Y_i = \text{dropout}\bigg(\text{softmax}\bigg(\frac{(XQ_i)(XK_i)^\top}{\sqrt{d/h}}\bigg)\bigg)(XV_i)
\end{equation}

最后，自注意力的输出是对各个注意力头的结果进行拼接后再进行线性变换得到的：

\begin{equation}
Y = [Y_1;\dots;Y_h]A
\end{equation}

其中$A \in\mathbb{R}^{d\times d}$，且$[Y_1;\dots;Y_h]\in\mathbb{R}^{\ell \times d}$。

请在文件`cs231n/transformer_layers.py`的`MultiHeadAttention`类中实现多头缩放点积注意力。下面的代码将检查你的实现。相对误差应小于`e-3`。

In [ ]:
torch.manual_seed(231)  # 设置随机种子，保证结果可复现

# 选择不同的维度以便于调试：
# 具体来说，以下值分别对应 N=1, H=2, T=3, E//H=4, E=8
batch_size = 1  # 批次大小
sequence_length = 3  # 序列长度
embed_dim = 8  # 嵌入维度
attn = MultiHeadAttention(embed_dim, num_heads=2)  # 初始化多头注意力层，嵌入维度为8，头数为2

# 自注意力计算
data = torch.randn(batch_size, sequence_length, embed_dim)  # 生成随机输入数据
self_attn_output = attn(query=data, key=data, value=data)  # 执行自注意力计算（Q、K、V均为同一输入）

# 带掩码的自注意力计算
mask = torch.randn(sequence_length, sequence_length) < 0.5  # 生成随机掩码（True/False矩阵）
masked_self_attn_output = attn(query=data, key=data, value=data, attn_mask=mask)  # 执行带掩码的自注意力计算

# 使用两个不同输入的注意力计算
other_data = torch.randn(batch_size, sequence_length, embed_dim)  # 生成另一个随机输入数据
attn_output = attn(query=data, key=other_data, value=other_data)  # 执行注意力计算（Q与K、V来自不同输入）

# 预期的自注意力输出结果
expected_self_attn_output = np.asarray([[
[-0.2494,  0.1396,  0.4323, -0.2411, -0.1547,  0.2329, -0.1936,
          -0.1444],
         [-0.1997,  0.1746,  0.7377, -0.3549, -0.2657,  0.2693, -0.2541,
          -0.2476],
         [-0.0625,  0.1503,  0.7572, -0.3974, -0.1681,  0.2168, -0.2478,
          -0.3038]]])

# 预期的带掩码自注意力输出结果
expected_masked_self_attn_output = np.asarray([[
[-0.1347,  0.1934,  0.8628, -0.4903, -0.2614,  0.2798, -0.2586,
          -0.3019],
         [-0.1013,  0.3111,  0.5783, -0.3248, -0.3842,  0.1482, -0.3628,
          -0.1496],
         [-0.2071,  0.1669,  0.7097, -0.3152, -0.3136,  0.2520, -0.2774,
          -0.2208]]])

# 预期的跨输入注意力输出结果
expected_attn_output = np.asarray([[
[-0.1980,  0.4083,  0.1968, -0.3477,  0.0321,  0.4258, -0.8972,
          -0.2744],
         [-0.1603,  0.4155,  0.2295, -0.3485, -0.0341,  0.3929, -0.8248,
          -0.2767],
         [-0.0908,  0.4113,  0.3017, -0.3539, -0.1020,  0.3784, -0.7189,
          -0.2912]]])

# 计算并打印实际输出与预期输出之间的相对误差
print('自注意力输出误差: ', rel_error(expected_self_attn_output, self_attn_output.detach().numpy()))
print('带掩码自注意力输出误差: ', rel_error(expected_masked_self_attn_output, masked_self_attn_output.detach().numpy()))
print('跨输入注意力输出误差: ', rel_error(expected_attn_output, attn_output.detach().numpy()))

# 位置编码

尽管Transformer能够轻松关注输入的任何部分，但注意力机制本身并没有 token 顺序的概念。然而，对于许多任务（尤其是自然语言处理），token 的相对顺序非常重要。为了弥补这一点，作者们在单个词 token 的嵌入中加入了位置编码。

我们来定义一个矩阵 $P \in \mathbb{R}^{l\times d}$，其中 $P_{ij} = $

$$
\begin{cases}
\text{sin}\left(i \cdot 10000^{-\frac{j}{d}}\right) & \text{if j is even} \\
\text{cos}\left(i \cdot 10000^{-\frac{(j-1)}{d}}\right) & \text{otherwise} \\
\end{cases}
$$

# 位置编码

我们不会直接将输入$X \in \mathbb{R}^{l\times d}$传入网络，而是传入$X + P$。

请在`cs231n/transformer_layers.py`的`PositionalEncoding`中实现这个层。完成后，运行以下代码对实现进行简单测试。你应该会看到误差在`e-3`量级或更小。

In [ ]:
torch.manual_seed(231)  # 设置随机种子，确保结果可复现

batch_size = 1  # 批次大小
sequence_length = 2  # 序列长度
embed_dim = 6  # 嵌入维度
data = torch.randn(batch_size, sequence_length, embed_dim)  # 生成随机输入数据

pos_encoder = PositionalEncoding(embed_dim)  # 初始化位置编码层，嵌入维度为6
output = pos_encoder(data)  # 对输入数据应用位置编码（输入数据 + 位置编码矩阵）

# 预期的位置编码输出结果
expected_pe_output = np.asarray([[[-1.2340,  1.1127,  1.6978, -0.0865, -0.0000,  1.2728],
                                  [ 0.9028, -0.4781,  0.5535,  0.8133,  1.2644,  1.7034]]])

# 计算并打印实际输出与预期输出之间的相对误差
print('位置编码输出误差: ', rel_error(expected_pe_output, output.detach().numpy()))

# 内联问题1

在设计我们上面介绍的缩放点积注意力时，做出了几个关键的设计决策。请解释以下选择为何是有利的：
1. 使用多个注意力头而非单个注意力头。
2. 在应用softmax函数之前除以$\sqrt{d/h}$。回想一下，$d$是特征维度，$h$是注意力头的数量。
3. 对注意力操作的输出进行线性变换。

每个选择只需用一两句话回答，但务必具体说明：如果没有每个给定的实现细节，会发生什么情况；为什么这种情况是不理想的；以及所提出的实现如何改善这种情况。

**你的答案：**


# Transformer解码器块

Transformer解码器层由三个模块组成：(1) 自注意力模块，用于处理输入的向量序列；(2) 交叉注意力模块，用于基于可用的上下文进行处理（在我们的例子中即图像特征）；(3) 前馈模块，用于独立处理序列中的每个向量。请在`cs231n/transformer_layers.py`中完成`TransformerDecoderLayer`的实现，并在下面进行测试。相对误差应小于1e-6。

Transformer解码器层有三个主要组件：(1) 自注意力模块，处理输入的向量序列；(2) 交叉注意力模块，整合额外的上下文（例如在我们的例子中是图像特征）；(3) 前馈模块，独立处理序列中的每个向量。请在`cs231n/transformer_layers.py`中完成`TransformerDecoderLayer`的实现，并在下面进行测试。相对误差应小于1e-6。

In [ ]:
torch.manual_seed(231)  # 设置PyTorch随机种子，确保结果可复现
np.random.seed(231)  # 设置NumPy随机种子，确保结果可复现

# 定义维度参数：
# N=批次大小，T=目标序列长度，TM=记忆序列长度，D=特征维度
N, T, TM, D = 1, 4, 5, 12

# 初始化Transformer解码器层：特征维度D=12，注意力头数=2，前馈网络维度=4*D=48
decoder_layer = TransformerDecoderLayer(D, 2, 4*D)
tgt = torch.randn(N, T, D)  # 生成目标序列输入（随机张量）
memory = torch.randn(N, TM, D)  # 生成记忆序列（如图像特征，随机张量）
tgt_mask = torch.randn(T, T) < 0.5  # 生成目标序列的掩码（True/False矩阵）

output = decoder_layer(tgt, memory, tgt_mask)  # 解码器层前向传播计算输出

# 预期的解码器层输出结果
expected_output = np.asarray([
    [[ 1.1464597, -0.32541496,  0.39171425, -0.39425734,  0.62471056,
      -1.8665842, -0.12977494, -1.6609063, -0.5620399,  0.45006236,
       1.6086785,  0.7173523],
     [-0.6703264,  0.34731007, -0.01452054, -0.0500976,  0.9617562,
      -0.91788256,  0.5138556, -1.5247818,  2.0940537, -1.0386938,
       1.0333964, -0.7340692],
     [-1.1966342,  0.78882384,  0.1765188,  0.04164891,  1.9480462,
      -0.94358695,  0.83423877, -0.44660965,  1.1469632, -1.6658922,
      -0.27915588, -0.4043607],
     [-0.96863323,  0.10736976, -0.18560877, -0.86474127, -0.12873,
       0.36593518,  0.9634492, -0.9432319,  1.4652547,  1.2200648,
       0.9218512, -1.9529796]]
])

# 计算并打印实际输出与预期输出的相对误差
print('误差: ', rel_error(expected_output, output.detach().numpy()))

# 用于图像描述生成的Transformer

既然你已经实现了前面的各个层，现在可以将它们组合起来，构建一个基于Transformer的图像描述生成模型。打开文件`cs231n/classifiers/transformer.py`，查看`CaptioningTransformer`类。

实现该类的`forward`函数。完成后，运行以下代码，通过一个小的测试用例检查你的前向传播；你应该会看到误差在`e-5`量级或更小。

In [ ]:
torch.manual_seed(231)  # 设置PyTorch随机种子，保证结果可复现
np.random.seed(231)  # 设置NumPy随机种子，保证结果可复现

# 定义参数：
# N=批次大小，D=图像特征维度，W=词向量维度
# word_to_idx=词到索引的映射字典，V=词汇表大小，T=字幕序列长度
N, D, W = 4, 20, 30
word_to_idx = {'<NULL>': 0, 'cat': 2, 'dog': 3}  # 词与索引的映射（<NULL>为填充符）
V = len(word_to_idx)  # 词汇表大小
T = 3  # 字幕序列长度

# 初始化基于Transformer的图像描述生成模型
transformer = CaptioningTransformer(
    word_to_idx,  # 词到索引的映射
    input_dim=D,  # 图像特征输入维度
    wordvec_dim=W,  # 词向量维度
    num_heads=2,  # 注意力头数
    num_layers=2,  # Transformer解码器层数
    max_length=30  # 生成字幕的最大长度
)

features = torch.randn(N, D)  # 生成随机图像特征（批次大小N，维度D）
captions = torch.randint(0, V, (N, T))  # 生成随机字幕序列（每个元素为词汇表中的索引）

scores = transformer(features, captions)  # 模型前向传播，得到预测分数

# 预期的预测分数结果
expected_scores = np.asarray([
    [[ 0.48119992, -0.24859881, -0.7489549 ],
     [ 0.20380056,  0.08959456, -0.89954275],
     [ 0.21135767, -0.17083111, -0.62508506]],

    [[ 0.49413955, -0.50489324, -0.79341394],
     [ 0.87452495, -0.4392967 , -1.1513498 ],
     [ 0.2547267 , -0.26321974, -0.93643296]],

    [[ 0.70437765, -0.5729916 , -0.7946507 ],
     [ 0.18345363, -0.31752932, -1.7304884 ],
     [ 0.61473167, -0.82634443, -1.2179294 ]],

    [[ 0.5163983 , -0.7899667 , -1.0383208 ],
     [ 0.28063023, -0.3603301 , -1.5435203 ],
     [ 0.7222998 , -0.71457165, -0.76669186]]
])

# 计算并打印实际预测分数与预期结果的相对误差
print('分数误差: ', rel_error(expected_scores, scores.detach().numpy()))

# 在小数据集上对Transformer图像生成模型进行过拟合

运行以下代码，在我们之前用于RNN的相同小数据集上对基于Transformer的字幕生成模型进行过拟合。

In [ ]:
torch.manual_seed(231)  # 设置PyTorch随机种子，确保结果可复现
np.random.seed(231)  # 设置NumPy随机种子，确保结果可复现

# 加载COCO数据集，最多加载50个训练样本
data = load_coco_data(max_train=50)

# 初始化基于Transformer的图像描述生成模型
transformer = CaptioningTransformer(
          word_to_idx=data['word_to_idx'],  # 词到索引的映射字典
          input_dim=data['train_features'].shape[1],  # 图像特征的维度
          wordvec_dim=256,  # 词向量的维度
          num_heads=2,  # 注意力头的数量
          num_layers=2,  # Transformer解码器的层数
          max_length=30  # 生成描述的最大长度
        )


# 初始化Transformer图像描述生成模型的求解器
transformer_solver = CaptioningSolverTransformer(transformer, data, idx_to_word=data['idx_to_word'],
           num_epochs=100,  # 训练的轮数
           batch_size=25,  # 批次大小
           learning_rate=0.001,  # 学习率
           verbose=True,  # 显示详细训练信息
           print_every=10,  # 每10次迭代打印一次信息
         )

# 训练模型
transformer_solver.train()

# 绘制训练损失曲线
plt.plot(transformer_solver.loss_history)
plt.xlabel('迭代次数')
plt.ylabel('损失值')
plt.title('训练损失历史')
plt.show()

打印最终的训练损失。你应该会看到最终损失小于0.05。

In [ ]:
print('最终损失: ', transformer_solver.loss_history[-1])  # 打印训练过程中的最终损失值（取损失历史列表的最后一个元素）

# Transformer在测试时的采样

采样代码已经为你编写好了。你只需运行以下代码，与之前RNN的结果进行比较。和之前一样，鉴于我们训练所用的数据量极少，训练集上的结果应该会比验证集上的结果好得多。

In [ ]:
# 如果你遇到错误，可能是URL已失效，无需担心！
# 你可以根据需要多次重新采样
for split in ['train', 'val']:  # 遍历训练集和验证集
    # 从指定数据集（训练集或验证集）中随机采样一个小批次数据，批次大小为2
    minibatch = sample_coco_minibatch(data, split=split, batch_size=2)
    gt_captions, features, urls = minibatch  # 解包得到真实字幕、图像特征和图像URL
    # 将真实字幕从索引序列解码为文本
    gt_captions = decode_captions(gt_captions, data['idx_to_word'])

    # 使用Transformer模型对图像特征进行采样，生成字幕，最大长度为30
    sample_captions = transformer.sample(features, max_length=30)
    # 将生成的字幕从索引序列解码为文本
    sample_captions = decode_captions(sample_captions, data['idx_to_word'])

    # 遍历每个样本的真实字幕、生成字幕和图像URL
    for gt_caption, sample_caption, url in zip(gt_captions, sample_captions, urls):
        img = image_from_url(url)  # 从URL加载图像
        if img is None: continue  # 跳过无法加载的图像
        plt.imshow(img)  # 显示图像
        # 设置图像标题，包含数据集划分、生成的字幕和真实字幕
        plt.title('%s\n%s\n真实字幕:%s' % (split, sample_caption, gt_caption))
        plt.axis('off')  # 关闭坐标轴显示
        plt.show()  # 显示图像及标题

# 视觉Transformer（ViT）

[Dosovitskiy et. al.](https://arxiv.org/abs/2010.11929)表明，将Transformer模型应用于图像补丁序列（称为视觉Transformer）不仅能取得令人瞩目的性能，而且在大型数据集上训练时，其扩展性比卷积神经网络更优。我们将使用现有的Transformer组件实现来构建一个视觉Transformer版本，并在CIFAR-10数据集上对其进行训练。

视觉Transformer会将输入图像转换为固定大小的补丁序列，并将每个补丁嵌入到一个潜在向量中。请在`cs231/transformer_layers.py`中完成`PatchEmbedding`的实现，并在下面进行测试。你应该会看到相对误差小于1e-4。

In [ ]:
from cs231n.transformer_layers import PatchEmbedding  # 导入PatchEmbedding类

torch.manual_seed(231)  # 设置PyTorch随机种子，确保结果可复现
np.random.seed(231)  # 设置NumPy随机种子，确保结果可复现

# 定义参数：
# N=批次大小，HW=图像的高度和宽度（假设图像是正方形），PS=补丁大小，D=嵌入维度
N = 2
HW = 16
PS = 8
D = 8

# 初始化补丁嵌入层：
# 图像大小为HW×HW，补丁大小为PS×PS，嵌入维度为D
patch_embedding = PatchEmbedding(
    img_size=HW,
    patch_size=PS,
    embed_dim=D
)

# 生成随机输入图像：批次大小N，3个通道（RGB），图像尺寸HW×HW
x = torch.randn(N, 3, HW, HW)
output = patch_embedding(x)  # 对输入图像进行补丁嵌入操作


# 预期的补丁嵌入输出结果
expected_output = np.asarray([
        [[-0.6312704 ,  0.02531429,  0.6112642 , -0.49089882,
          0.01412961, -0.6959372 , -0.32862484, -0.45402682],
        [ 0.18816411, -0.08142513, -0.9829535 , -0.23975623,
         -0.23109074,  0.97950286, -0.40997326,  0.7457837 ],
        [ 0.01810865,  0.15780598, -0.91804236,  0.36185235,
          0.8379501 ,  1.0191797 , -0.29667392,  0.20322265],
        [-0.18697818, -0.45137224, -0.40339014, -1.4381214 ,
         -0.43450755,  0.7651071 , -0.83683825, -0.16360264]],

       [[-0.39786366,  0.16201034, -0.19008337, -1.0602452 ,
         -0.28693503,  0.09791763,  0.26614824,  0.41781986],
        [ 0.35146567, -0.4469593 , -0.1841726 ,  0.45757473,
         -0.61304873, -0.29104248, -0.16124889, -0.14987172],
        [-0.2996967 ,  0.27353522, -0.09929767,  0.01973832,
         -1.2312065 , -0.6374332 , -0.22963578,  0.55696607],
        [-0.93818814,  0.02465284, -0.21117875,  1.1860403 ,
         -0.06137538, -0.21062079, -0.094347  ,  0.50032747]]])

# 计算并打印实际输出与预期输出的相对误差
print('误差: ', rel_error(expected_output, output.detach().numpy()))

补丁向量序列由Transformer编码器层处理，每个编码器层包含一个自注意力模块和一个前馈模块。由于所有向量都会相互关注，注意力掩码并非绝对必要。但为了保持一致性，我们仍然会实现它。

请在`cs231n/transformer_layers.py`中实现`TransformerEncoderLayer`，并在下面进行测试。你应该会看到相对误差小于1e-6。


In [ ]:
torch.manual_seed(231)  # 设置PyTorch随机种子，确保结果可复现
np.random.seed(231)  # 设置NumPy随机种子，确保结果可复现

from cs231n.transformer_layers import TransformerEncoderLayer  # 导入Transformer编码器层类

# 定义维度参数：
# N=批次大小，T=序列长度，TM=（未使用，占位），D=特征维度
N, T, TM, D = 1, 4, 5, 12

# 初始化Transformer编码器层：特征维度D=12，注意力头数=2，前馈网络维度=4*D=48
encoder_layer = TransformerEncoderLayer(D, 2, 4*D)
x = torch.randn(N, T, D)  # 生成随机输入序列（批次大小N，长度T，维度D）
x_mask = torch.randn(T, T) < 0.5  # 生成输入序列的掩码（T×T的True/False矩阵）

output = encoder_layer(x, x_mask)  # 编码器层前向传播计算输出

# 预期的编码器层输出结果
expected_output = np.asarray([
    [[-0.43529928, -0.204897, 0.45693663, -1.1355408, 1.8000772,
      0.24467856, 0.8525885, -0.53586316, -1.5606489, -1.207276,
      1.3986266, 0.3266182],
     [0.06928468, 1.1030475, -0.9902548, -0.34333378, -2.1073136,
      1.1960536, 0.16573538, -1.1772276, 1.2644588, -0.27311313,
      0.29650143, 0.7961618],
     [0.28310525, 0.69066685, -1.2264299, 1.0175265, -2.0517688,
     -0.10330413, -0.5355796, -0.2696466, 0.13948536, 2.0408154,
      0.27095756, -0.25582793],
     [-0.58568114, 0.8019579, -0.9128079, -1.6816932, 1.1572194,
      0.39162305, 0.58195484, 0.7043353, -1.27042, -1.1870497,
      0.9784279, 1.0221335]]
])

# 计算并打印实际输出与预期输出的相对误差
print('误差: ', rel_error(expected_output, output.detach().numpy()))

查看`cs231n/classifiers/transformer.py`中的`VisionTransformer`实现。

在分类任务中，ViT将输入图像划分为补丁，并使用Transformer处理补丁向量序列。最后，所有补丁向量经过平均池化后用于预测图像类别。我们将使用相同的一维正弦位置编码来注入顺序信息，不过二维正弦编码和学习型位置编码也是有效的选择。

完成ViT的前向传播并在下面进行测试。你应该会看到相对误差小于1e-6。

In [ ]:
torch.manual_seed(231)  # 设置PyTorch随机种子，确保结果可复现
np.random.seed(231)  # 设置NumPy随机种子，确保结果可复现
from cs231n.classifiers.transformer import VisionTransformer  # 导入视觉Transformer类

imgs = torch.randn(3, 3, 32, 32)  # 生成随机输入图像：3个样本，3个通道（RGB），32×32像素
transformer = VisionTransformer()  # 初始化视觉Transformer模型
scores = transformer(imgs)  # 模型前向传播，得到分类分数

# 预期的分类分数结果
expected_scores = np.asarray(
    [[-0.13013132,  0.13652277, -0.04656096, -0.16443546, -0.08946665,
        -0.10123537,  0.11047452,  0.01317241,  0.17256221,  0.16230097],
       [-0.11988413,  0.20006064, -0.04028708, -0.06937674, -0.07828291,
        -0.13545093,  0.18698244,  0.01878054,  0.14309685,  0.03245382],
       [-0.11540816,  0.21416159, -0.07740889, -0.08336161, -0.1645808 ,
        -0.12318538,  0.18035144,  0.05492767,  0.15997584,  0.12134959]])

# 计算并打印实际分类分数与预期结果的相对误差
print('分类分数误差: ', rel_error(expected_scores, scores.detach().numpy()))


我们将首先通过在一个训练批次上对模型进行过拟合来验证我们的实现。

In [ ]:
from torchvision import transforms  # 导入torchvision的 transforms 模块，用于数据预处理
from torchvision.datasets import CIFAR10  # 导入CIFAR10数据集类
from tqdm.auto import tqdm  # 导入tqdm库，用于显示进度条
from torch.utils.data import DataLoader  # 导入DataLoader，用于数据加载

# 加载CIFAR10训练集：根目录为'data'，训练模式，将数据转换为Tensor格式，自动下载数据集
train_data = CIFAR10(root='data', train=True, transform=transforms.ToTensor(), download=True)
# 加载CIFAR10测试集：根目录为'data'，测试模式，将数据转换为Tensor格式，自动下载数据集
test_data = CIFAR10(root='data', train=False, transform=transforms.ToTensor(), download=True)

In [ ]:
learning_rate = 1e-4  # 可以调整学习率进行实验
weight_decay = 1.e-4  # 可以调整权重衰减系数进行实验


# 从训练数据加载器中获取一个批次的数据（批次大小64，不打乱顺序）
batch = next(iter(DataLoader(train_data, batch_size=64, shuffle=False)))
model = VisionTransformer(dropout=0.0)  # 初始化视觉Transformer模型，不使用dropout
loss_criterion = torch.nn.CrossEntropyLoss()  # 定义交叉熵损失函数（用于分类任务）
# 定义Adam优化器，传入模型参数、学习率和权重衰减系数
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
model.train()  # 将模型设置为训练模式

epochs = 100  # 训练轮数
for epoch in range(epochs):
    imgs, target = batch  # 解包批次数据：图像和对应的标签
    out = model(imgs)  # 模型前向传播，得到预测输出
    loss = loss_criterion(out, target)  # 计算损失值

    optimizer.zero_grad()  # 清空优化器的梯度
    loss.backward()  # 反向传播计算梯度
    optimizer.step()  # 更新模型参数

    # 计算Top-1准确率（预测类别与真实类别一致的样本比例）
    top1 = (out.argmax(-1) == target).float().mean().item()
    # 每10轮打印一次训练信息
    if epoch % 10 == 0:
      print(f"[{epoch}/{epochs}] 损失值 {loss.item():.6f}，Top-1准确率: {top1:.3f}")


In [ ]:
# 你应该会得到1.00的完美准确率
print(f"在一个批次上对ViT进行过拟合。Top-1准确率: {top1}")

现在我们将在整个数据集上训练它。

In [ ]:
from cs231n.classification_solver_vit import ClassificationSolverViT  # 导入视觉Transformer的分类求解器类

####################################################################################
# 任务：通过调整模型架构和/或训练参数，训练一个视觉Transformer模型，使其在2个 epoch后  #
# 在CIFAR-10数据集上的测试准确率超过0.45。                                           #
#                                                                                  #
# 注意：如果想使用GPU运行时，请前往 `Runtime > Change runtime type`，将              #
# `Hardware accelerator` 设置为 `GPU`。这会重置Colab环境，因此之后需从开头           #
# 重新运行整个笔记本。                                                              #
####################################################################################


learning_rate = 1e-4  # 学习率
weight_decay = 0.0  # 权重衰减系数
batch_size = 64  # 批次大小
model = VisionTransformer()  # 初始化视觉Transformer模型（你可能需要修改默认参数）




################################################################################
#                                 代码结束部分                                  #
################################################################################

# 初始化分类求解器
solver = ClassificationSolverViT(
    train_data=train_data,  # 训练数据
    test_data=test_data,  # 测试数据
    model=model,  # 模型
    num_epochs = 2,  # 训练轮数（请勿修改）
    learning_rate = learning_rate,  # 学习率
    weight_decay = weight_decay,  # 权重衰减系数
    batch_size = batch_size,  # 批次大小
)

# 在可用的设备（GPU或CPU）上训练模型
solver.train('cuda' if torch.cuda.is_available() else 'cpu')



In [ ]:
print(f"测试集准确率: {solver.results['best_test_acc']}")  # 打印测试集上的最佳准确率（从求解器的结果字典中获取）

# 内联问题2

尽管视觉Transformer（ViT）近年来在大规模图像识别任务中取得了成功，但在较小的数据集上训练时，它们的表现往往落后于传统的卷积神经网络（CNN）。导致这种性能差距的潜在因素是什么？可以使用哪些技术来提高ViTs在小数据集上的性能？

**你的答案**：填写此处。



# 内联问题3

如果我们分别进行以下更改，视觉Transformer（ViT）中自注意力层的计算成本会如何变化？

（i）将隐藏维度增加一倍。
（ii）将输入图像的高度和宽度都增加一倍。
（iii）将补丁大小增加一倍。
（iv）将网络层数增加一倍。

**你的答案**：填写此处。

